**8. Проблема множественного тестирования. Задания.**

Задания по видео 42 - 46.


# Описание задачи

## Информация из предыдущих этапов

**Постановка кейса.**

В рамках практических заданий мы будем работать аналитиком на платформе по подаче объявлений. По результатам исследований мы выяснили, что недавно введенной функцией доставки пользуются редко. В рамках идей по исправлению ситуации была предложена инициатива выдать скидку в 20% на данную услугу, зачеркнув старую цену и написав рядом новую на карточках объявлений, в которых доступна доставка. Нам поставлена задача оценить данную инициативу.

**Метрики.**

По резульатам командного обсуждения было принято назначить целевой метрикой выручку. Прочие метрики, за которыми ваша команда планирует наблюдать: выручка с доставки, пользователи с доставкой, количество ошибок на экранах доставочный айтемов, количество заказов без доставки.

**Критерий.**

По результатам предыдущего исследования наш выбор пал на тест Стьюдента. Предполагаем, что будем использовать его.

**MDE, N, alpha, beta**

По результатам обсуждения с аналитиками и продуктами было принято решение остановиться на ошибках 1 и 2 рода в 1% при MDE в 1.4% для основной метрики. N = 5 mln в каждой группе, длительность эксперимента - 21 день.

**Разбиение на тест-контроль**

Мы реализовали функцию деления выборки на тест-контроль с помощью остатка от деления от хэша id юзера и соли эксперимента. При этом мы модифицировали функцию так, чтобы данные до начала эксперимента не отличались между собой статистически значимо ни по одной метрике.

**Анализ результатов.**

Ваш коллега проанализировал тест, и оказалось, что ни по 1 метрике не обнаружилось статистически значимой разницы. Хотя тестовая группа и выигрывала по целевой и прокси-метрикам, доверительный интервал перекрывал 0, p value был значительно выше 0.05.

Следующим этапом было решено проанализировать наиболее чувствительный сегмент. Ваши коллеги подготовили такую выборку, на которую тестовое воздействие должно было повлиять наибольшим образом. В нее попали те пользователи, что находятся в удаленых регионах и переходили на карточки объявлений с возможностью доставки. То есть этим юзерам актуальна доставка, и они точно видели тестовое воздействие (или потенциально увидели бы его, если речь идет о контроле). Анализ показал, что результат вроде и положительный, но на грании с тем, что вполне могло получиться и случайно.

Далее было решено использовать CUPED для повышения чувствительно, и этот метод показал себя отлично: теперь вы уверены в том, что тестовое воздействие оказало положительный эффект на анализируемый сегмент. Далее вы проанализировали применение стратификации и пост-стратицикации. Этот метод показал значительно более скромные результаты.

## Текущее задание

Ваша команда решила провести вторую итерацию теста, после того как ввела дополнительные улучшения. В частности, поменяла дизайн отоброжения скидки. Еще одной идией является анализ различных уровней скидок: 10, 20 и 30%. Вам предстоит проанализировать, какая ошибка 1-го рода будет в случае попарных сравнений всех вариантов при различных статистических поправках. Также ваша команда попросила вас проверить: какие результаты дадут эти поправки, если мы будем анализировать только 2 группы, но у нас будет 5 метрик. По результатам вашего исследования будет приниматься решение о дизайне теста. Удачи!

Суммарно можно получить более 10 баллов. Помимо непосредственно заданий есть скрытые баллы, которые можно получить, если писать хороший код и делать то, о чем говорилось на лекциях. При этом оценка будет рассчитываться из расчета, что 10 баллов - это 100% выполнения.

Вам понадобятся данные:
*   task_1.

# Множественная проверка: 4 группы и 1 метрика. 5 баллов

## Формулировка

Проведите 10 тысяч симуляций, в рамках которых разбейте датасет task_1 на 4 равные части и попарно сравните группы между собой по метрике revenue. Проанализируйте 4 стратегии: отсутствие поправок на множественное тестирование, поправку Бонферрони, поправку Холма и поправку Беньямини-Хохберга. Используйте для этого абсолютную версию t-теста и alpha = 0.05.

По результатам симуляций выведите таблицу, где индексы - типы поправок, а колонки - % отклоненных гипотез и % итераций с отклонением хотя бы 1 гипотезы.

Ответьте на вопросы:
* К каким последствиям приводит отсутствие применение каких либо поправок?
* Какая поправка является наиболее консервативной?
* От какакой поправки ожидается наибольшая мощность?

In [18]:
import pandas as pd
import numpy as np
from itertools import combinations
import seaborn as sns
from tqdm import tqdm
from scipy.stats import ttest_ind, norm
from sklearn.model_selection import train_test_split
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportion_confint
from collections import namedtuple

In [3]:
pre_data = pd.read_csv('task_2.csv')

In [4]:
def no_correction(p_values, alpha):
    """возвращает индексы отвергнутых гипотез"""
    m = len(p_values)
    rejected = []
    for i in range(len(p_values)):
        if p_values[i] < alpha:
            rejected.append(i)
    return rejected

def bonferroni(p_values, alpha):
    """возвращает индексы отвергнутых гипотез"""
    m = len(p_values)
    rejected = []
    for i in range(m):
        if p_values[i] < alpha / m:
            rejected.append(i)
    return rejected

def holm(p_values, alpha):
    """возвращает индексы отвергнутых гипотез"""
    m = len(p_values)
    rejected = []
    i = 1
    for p_idx in np.argsort(p_values):
        if p_values[p_idx] < alpha / (m + 1 - i):
            rejected.append(p_idx)
            i += 1
        else:
            break
    return rejected

def ben_hoh(p_values, alpha):
    """возвращает индексы отвергнутых гипотез"""
    m = len(p_values)
    rejected = []
    i = 1
    for p_idx in np.argsort(p_values):
        if p_values[p_idx] < alpha * i / m:
            rejected.append(p_idx)
            i += 1
        else:
            break
    return rejected

In [5]:
corrections = [
      no_correction,
      bonferroni,
      holm,
      ben_hoh,
  ]
alpha = 0.05
aa_research = {correction.__name__: [] for correction in corrections}
for _ in tqdm(range(10_000)):
    control_test_1, test_2_test_3 = train_test_split(pre_data['revenue'], test_size=0.5)
    control, test_1 = train_test_split(control_test_1, test_size=0.5)
    test_2, test_3 = train_test_split(test_2_test_3, test_size=0.5)
    p_values = []
    for pair in combinations([test_3, test_2, test_1, control], 2):
        p_v = ttest_ind(pair[0], pair[1]).pvalue
        p_values.append(p_v)
    for correction in corrections:
        rejected = correction(
            p_values=p_values,
            alpha=alpha,
        )
        aa_research[correction.__name__].append(len(rejected))
        
aa_research_pd = pd.DataFrame(aa_research)

100%|██████████| 10000/10000 [00:57<00:00, 174.72it/s]


In [8]:
mean_values = aa_research_pd.mean()
positive_proportion = (aa_research_pd > 0).mean()

result_df = pd.DataFrame({
    'Доля отклонённых гипотез': mean_values,
    'FWER': positive_proportion
})

result_df.index.name = 'метод'

result_df.style.format(
    '{:.3f}',
    ).highlight_null(
        color="gray",
    ).background_gradient(
        axis='columns',
        cmap='Reds',
        vmin=0.05,
        vmax=0.2
    )

,Доля отклонённых гипотез,FWER
метод,,
no_correction,0.292,0.198
bonferroni,0.048,0.040
holm,0.049,0.040
ben_hoh,0.056,0.040


### Выводы
**1) К каким последствиям приводит отсутствие применения каких либо поправок?**
- Отклоняется 29% гипотез, а ошибка 1 рода вырастает до 0,2. 
- Применение любой из поправок позволяет удержать ошибку первого рода ниже заданного уровня.

**2) Какая поправка является наиболее консервативной?**
- Самой консервативной является поправка Бонферрони - она отклоняет меньше всего гипотез о равенстве (4,8%)

**3) От какакой поправки ожидается наибольшая мощность?**
- От поправки Бенжамини-Хохберга, так как она отклонила больше всего гипотез о равенстве, не считая случая отсутствия поправок (5,6%)

# Множественная проверка: 2 группы и 5 метрик. 5 баллов

## Формулировка

Проведите аналогичный анализ для случаях 2-ух метрик и 5 гипотез.

In [43]:
metrics = [
    'revenue',
    'delivery_revenue',
    'has_delivery',
    'error_count',
    'order_wo_delivery'
]

In [53]:
corrections = [
      no_correction,
      bonferroni,
      holm,
      ben_hoh,
  ]
alpha = 0.05
aa_research = {correction.__name__: [] for correction in corrections}
for _ in tqdm(range(10_000)):
    p_values = []
    for metric in metrics:
        control, test = train_test_split(pre_data[metric], test_size=0.5)
        p_v = ttest_ind(control, test).pvalue
        p_values.append(p_v)
        for correction in corrections:
            rejected = correction(
                p_values=p_values,
                alpha=alpha,
            )
            aa_research[correction.__name__].append(len(rejected))
        
aa_research_pd = pd.DataFrame(aa_research)

100%|██████████| 10000/10000 [01:48<00:00, 92.40it/s]


In [54]:
mean_values = aa_research_pd.mean()
positive_proportion = (aa_research_pd > 0).mean()

result_df = pd.DataFrame({
    'Доля отклонённых гипотез': mean_values,
    'FWER': positive_proportion
})

result_df.index.name = 'метод'

result_df.style.format(
    '{:.3f}',
    ).highlight_null(
        color="gray",
    ).background_gradient(
        axis='columns',
        cmap='Reds',
        vmin=0.05,
        vmax=0.2
    )

,Доля отклонённых гипотез,FWER
метод,,
no_correction,0.154,0.143
bonferroni,0.049,0.048
holm,0.050,0.048
ben_hoh,0.051,0.048


### Вывод
- Поправки значительно снизили долю отклонённых гипотез о равенстве и ошибку 1 рода.
- Тем не менее, не можем использовать эти поправки для ситуации с множеством метрик, так как нарушается предпосылка о независимости наблюдений: метрики считаются по одним и тем же пользователям и скоррелированы между собой